# <font color='blue'>Data Science Academy - Python Fundamentos - Capítulo 12</font>

## Download: http://github.com/dsacademybr

## Mini-Projeto 4 - Inteligência Artificial na Agricultura 

**ATENÇÃO**

Este Mini-Projeto é um bônus com nível intermediário/avançado e o que apresentaremos aqui é apenas uma demonstração. Os conceitos necessários para executar este Mini-Projeto são estudados em detalhes na <a href="https://www.datascienceacademy.com.br/bundle/formacao-inteligencia-artificial">Formação Inteligência Artificial</a> e <a href="https://www.datascienceacademy.com.br/bundle/formacao-inteligencia-artificial-aplicada-a-medicina">Formação Inteligência Artificial Aplicada à Medicina</a> aqui na DSA.

![title](imagens/mini-projeto4.png)

## Definição do Problema

Acesse o manual em pdf no Capítulo 12 do curso.

## Fonte de Dados

Acesse o manual em pdf no Capítulo 12 do curso.

## Instalando e Carregando Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.7


In [2]:
# Instala o TensorFlow
#!pip install -q tensorflow==2.5

In [3]:
# Imports
import sklearn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall

In [4]:
# Seed para reprodutibilidade
tf.random.set_seed(4)

## Carregando os Dados (Imagens)

In [5]:
# Diretório atual
diretorio_atual = Path.cwd()
print(diretorio_atual)

C:\Users\welli\Documents\Projects\python-projects\python-fundamentals-data-analysis-3.0\PythonFundamentos\Cap12


In [6]:
# Caminho para os dados de treino
caminho_dados_treino = Path("fruits-360/Training")

In [7]:
# Caminho para os dados de teste
caminho_dados_teste = Path("fruits-360/Test")

In [8]:
# Listando o conteúdo da pasta
imagens_treino = list(caminho_dados_treino.glob("*/*"))

In [9]:
# Visualiza uma amostra da lista
imagens_treino[925:936]

[WindowsPath('fruits-360/Training/Apple Crimson Snow/r_90_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_91_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_92_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_93_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_94_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_95_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_96_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_97_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_98_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_99_100.jpg'),
 WindowsPath('fruits-360/Training/Apple Crimson Snow/r_9_100.jpg')]

In [10]:
# Expressão lambda que extrai apenas o valor com o caminho de cada imagem
imagens_treino = list(map(lambda x: str(x), imagens_treino))

In [11]:
# Visualiza uma amostra da lista
imagens_treino[925:936]

['fruits-360\\Training\\Apple Crimson Snow\\r_90_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_91_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_92_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_93_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_94_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_95_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_96_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_97_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_98_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_99_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_9_100.jpg']

In [12]:
# Total de imagens de treino
len(imagens_treino)

67692

## Pré-Processamento dos Dados

In [13]:
# Função que obtém o label de cada imagem
def extrai_label(caminho_imagem):
    return caminho_imagem.split("/")[-1]

In [14]:
# Aplica a função
imagens_treino_labels = list(map(lambda x: extrai_label(x), imagens_treino))

In [15]:
# Visualiza uma amostra
imagens_treino_labels[840:846]

['fruits-360\\Training\\Apple Crimson Snow\\r_238_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_239_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_240_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_241_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_242_100.jpg',
 'fruits-360\\Training\\Apple Crimson Snow\\r_243_100.jpg']

> Label encoding (convertendo string para valor numérico)

In [16]:
# Cria o objeto
encoder = LabelEncoder()

In [17]:
# Aplica o fit_transform
imagens_treino_labels = encoder.fit_transform(imagens_treino_labels)

In [18]:
# Visualiza uma amostra
imagens_treino_labels[840:846]

array([840, 841, 842, 843, 844, 845], dtype=int64)

In [19]:
# Aplicamos One-Hot-Encoding nos labels
imagens_treino_labels = tf.keras.utils.to_categorical(imagens_treino_labels)

In [20]:
# Visualiza uma amostra
imagens_treino_labels[840:846]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [21]:
# Dividimos os dados de treino em duas amostras, treino e validação
X_treino, X_valid, y_treino, y_valid = train_test_split(imagens_treino, imagens_treino_labels)

In [22]:
X_treino[15:18]

['fruits-360\\Training\\Pepper Orange\\r2_136_100.jpg',
 'fruits-360\\Training\\Tomato Heart\\r2_175_100.jpg',
 'fruits-360\\Training\\Cherry Wax Red\\r_16_100.jpg']

In [23]:
y_treino[15:18]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Dataset Augmentation

In [24]:
# Redimensionamento de todas as imagens para 224 x 224
img_size = 224
resize = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Resizing(img_size, img_size)])

In [25]:
# Cria o objeto para dataset augmentation
data_augmentation = tf.keras.Sequential([RandomFlip("horizontal"),
                                         RandomRotation(0.2),
                                         RandomZoom(height_factor = (-0.3,-0.2)) ])

## Preparando os Dados

In [26]:
# Hiperparâmnetros
batch_size = 32
autotune = tf.data.experimental.AUTOTUNE

In [27]:
# Função para carregar e transformar as imagens
def carrega_transforma(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    return image, label

In [28]:
# Função para preparar os dados noo formato do TensorFlow
def prepara_dataset(path, labels, train = True):

    # Prepara os dados
    image_paths = tf.convert_to_tensor(path)
    labels = tf.convert_to_tensor(labels)
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    dataset = dataset.map(lambda image, label: carrega_transforma(image, label)) 
    dataset = dataset.map(lambda image, label: (resize(image), label), num_parallel_calls = autotune)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)

    # Se train = True aplica dataset augmentation
    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label), num_parallel_calls = autotune)
  
    # Se train = False repete sobre o dataset e retorna
    dataset = dataset.repeat()

    return dataset

In [29]:
# Cria o dataset de treino
dataset_treino = prepara_dataset(X_treino, y_treino)

In [ ]:
# Shape
imagem, label = next(iter(dataset_treino))
print(imagem.shape)
print(label.shape)

In [ ]:
# Vamos visualizar uma imagem e um label
print(encoder.inverse_transform(np.argmax(label, axis = 1))[0])
plt.imshow((imagem[0].numpy()/255).reshape(224,224,3))

In [ ]:
# Cria o dataset de validação
dataset_valid = prepara_dataset(X_valid, y_valid, train = False)

In [ ]:
# Shape
imagem, label = next(iter(dataset_valid))
print(imagem.shape) 
print(label.shape)

## Construção do Modelo

In [ ]:
# Carregando um modelo pré-treinado
modelo_pre = EfficientNetB3(input_shape = (224,224,3), include_top = False)

In [ ]:
# Adicionando nossas próprias camadas ao modelo_pre
modelo = tf.keras.Sequential([modelo_pre,
                              tf.keras.layers.GlobalAveragePooling2D(),
                              tf.keras.layers.Dense(131, activation = 'softmax')])

In [ ]:
# Sumário do modelo
modelo.summary()

In [ ]:
# Hiperparâmetros
lr = 0.001
beta1 = 0.9
beta2 = 0.999
ep = 1e-07

In [ ]:
# Compilação do modelo
modelo.compile(optimizer = Adam(learning_rate = lr, 
                                beta_1 = beta1, 
                                beta_2 = beta2, 
                                epsilon = ep),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy', Precision(name = 'precision'), Recall(name = 'recall')])

> Vamos treinar o modelo por apenas uma época e verificar as métricas.

**Nota**: Se você treinar o modelo em um computador sem GPU o tempo de treinamento será bastante alto, provavelmente de muitas horas. Pratique a paciência e aguarde.

In [ ]:
%%time
history = modelo.fit(dataset_treino,
                     steps_per_epoch = len(X_treino)//batch_size,
                     epochs = 1,
                     validation_data = dataset_valid,
                     validation_steps = len(y_treino)//batch_size)

> Vamos treinar o modelo por mais 6 épocas a fim de melhorar a performance e aplicar algumas técnicas para evitar overfitting.

In [ ]:
# Não precisamos mais do modelo_pre
modelo.layers[0].trainable = False

In [ ]:
# Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint("modelo/melhor_modelo.h5", 
                                                verbose = 1, 
                                                save_best = True, 
                                                save_weights_only = True)

In [ ]:
# Early stop
early_stop = tf.keras.callbacks.EarlyStopping(patience = 4) 

In [ ]:
# Sumário
modelo.summary()

In [ ]:
%%time
history = modelo.fit(dataset_treino,
                     steps_per_epoch = len(X_treino)//batch_size,
                     epochs = 6,
                     validation_data = dataset_valid,
                     validation_steps = len(y_treino)//batch_size,
                     callbacks = [checkpoint, early_stop])

## Avaliação do Modelo

In [ ]:
# Para carregar os pesos, precisamos descongelar as camadas
modelo.layers[0].trainable = True

In [ ]:
# Carrega os pesos do ponto de verificação e reavalie
modelo.load_weights("modelo/melhor_modelo.h5")

> Carregamos os dados de teste.

In [ ]:
# Carregando e preparando os dados de teste
camninho_imagens_teste = list(caminho_dados_teste.glob("*/*"))
imagens_teste = list(map(lambda x: str(x), camninho_imagens_teste))
imagens_teste_labels = list(map(lambda x: extrai_label(x), imagens_teste))
imagens_teste_labels = encoder.fit_transform(imagens_teste_labels)
imagens_teste_labels = tf.keras.utils.to_categorical(imagens_teste_labels)
test_image_paths = tf.convert_to_tensor(imagens_teste)
test_image_labels = tf.convert_to_tensor(imagens_teste_labels)

In [ ]:
# Função para decode das imagens
def decode_imagens(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [224,224], method = "bilinear")
    return image, label

In [ ]:
# Cria o dataset de teste
dataset_teste = (tf.data.Dataset
                 .from_tensor_slices((imagens_teste, imagens_teste_labels))
                 .map(decode_imagens)
                 .batch(batch_size))

In [ ]:
# Shape
imagem, label = next(iter(dataset_teste))
print(imagem.shape)
print(label.shape)

In [ ]:
# Visualiza uma imagem de teste
print(encoder.inverse_transform(np.argmax(label, axis = 1))[0])
plt.imshow((imagem[0].numpy()/255).reshape(224,224,3))

In [ ]:
# Avalia o modelo
loss, acc, prec, rec = modelo.evaluate(dataset_teste)

In [ ]:
print("Acurácia: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)

## Previsões com o Modelo Treinado

In [ ]:
# Função para carregar uma nova imagem
def carrega_nova_imagem(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [224,224], method = "bilinear")
    plt.imshow(image.numpy()/255)
    image = tf.expand_dims(image, 0) 
    return image

In [ ]:
# Função para fazer previsões
def faz_previsao(image_path, model, enc):
    image = carrega_nova_imagem(image_path)
    prediction = model.predict(image)
    pred = np.argmax(prediction, axis = 1) 
    return enc.inverse_transform(pred)[0] 

In [ ]:
# Previsão
faz_previsao("imagens/imagem1.jpg", modelo, encoder)

In [ ]:
# Previsão
faz_previsao("imagens/imagem2.jpg", modelo, encoder)

In [ ]:
# Previsão
faz_previsao("imagens/imagem3.jpg", modelo, encoder)

In [ ]:
# Previsão
faz_previsao("imagens/imagem4.jpg", modelo, encoder)

In [ ]:
# Previsão
faz_previsao("imagens/imagem5.jpg", modelo, encoder)

# Fim